<a href="https://colab.research.google.com/github/Toxxi-frank/Deeplearning/blob/main/Phan_load_mia_mai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import json để đọc file; text và sequence để chuyển word2vec và add padding

In [ ]:
import tensorflow as tf
import json
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Flatten

In [ ]:
vocab_size = 10000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000

In [ ]:
!wget --no-check-certificate \https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json \
-O /tmp/sarcasm.json 

--2021-06-16 08:47:07--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.203.128, 142.250.98.128, 64.233.170.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘/tmp/sarcasm.json’

/tmp/sarcasm.json   100%[===================>]   5.38M  --.-KB/s    in 0.06s   

2021-06-16 08:47:07 (94.6 MB/s) - ‘/tmp/sarcasm.json’ saved [5643545/5643545]



In [ ]:
with open('/tmp/sarcasm.json', 'r') as f:
  datastore = json.load(f)

file gồm 3 phần: article_link, headline, is_sarcastic

In [ ]:
datastore

[{'article_link': 'https://www.huffingtonpost.com/entry/versace-black-code_us_5861fbefe4b0de3a08f600d5',
  'headline': "former versace store clerk sues over secret 'black code' for minority shoppers",
  'is_sarcastic': 0},
 {'article_link': 'https://www.huffingtonpost.com/entry/roseanne-revival-review_us_5ab3a497e4b054d118e04365',
  'headline': "the 'roseanne' revival catches up to our thorny political mood, for better and worse",
  'is_sarcastic': 0},
 {'article_link': 'https://local.theonion.com/mom-starting-to-fear-son-s-web-series-closest-thing-she-1819576697',
  'headline': "mom starting to fear son's web series closest thing she will have to grandchild",
  'is_sarcastic': 1},
 {'article_link': 'https://politics.theonion.com/boehner-just-wants-wife-to-listen-not-come-up-with-alt-1819574302',
  'headline': 'boehner just wants wife to listen, not come up with alternative debt-reduction ideas',
  'is_sarcastic': 1},
 {'article_link': 'https://www.huffingtonpost.com/entry/jk-rowling-w

In [ ]:
sentences = []
labels = []

for item in datastore:
  sentences.append(item['headline'])
  labels.append(item['is_sarcastic'])

In [ ]:
import numpy as np
labels = np.array(labels)

In [ ]:
sentences

["former versace store clerk sues over secret 'black code' for minority shoppers",
 "the 'roseanne' revival catches up to our thorny political mood, for better and worse",
 "mom starting to fear son's web series closest thing she will have to grandchild",
 'boehner just wants wife to listen, not come up with alternative debt-reduction ideas',
 'j.k. rowling wishes snape happy birthday in the most magical way',
 "advancing the world's women",
 'the fascinating case for eating lab-grown meat',
 'this ceo will send your kids to school, if you work for his company',
 'top snake handler leaves sinking huckabee campaign',
 "friday's morning email: inside trump's presser for the ages",
 'airline passengers tackle man who rushes cockpit in bomb threat',
 'facebook reportedly working on healthcare features and apps',
 "north korea praises trump and urges us voters to reject 'dull hillary'",
 "actually, cnn's jeffrey lord has been 'indefensible' for a while",
 'barcelona holds huge protest in su

In [ ]:
labels

array([0, 0, 1, ..., 0, 0, 0])

In [ ]:
labels[13]

0

Chia thành 2 phần: training và validation

In [ ]:
training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]

training_labels = labels[0:training_size]
testing_labels = labels[training_size:]


In [ ]:
print(len(training_labels))

20000


In [ ]:
training_sentences

["former versace store clerk sues over secret 'black code' for minority shoppers",
 "the 'roseanne' revival catches up to our thorny political mood, for better and worse",
 "mom starting to fear son's web series closest thing she will have to grandchild",
 'boehner just wants wife to listen, not come up with alternative debt-reduction ideas',
 'j.k. rowling wishes snape happy birthday in the most magical way',
 "advancing the world's women",
 'the fascinating case for eating lab-grown meat',
 'this ceo will send your kids to school, if you work for his company',
 'top snake handler leaves sinking huckabee campaign',
 "friday's morning email: inside trump's presser for the ages",
 'airline passengers tackle man who rushes cockpit in bomb threat',
 'facebook reportedly working on healthcare features and apps',
 "north korea praises trump and urges us voters to reject 'dull hillary'",
 "actually, cnn's jeffrey lord has been 'indefensible' for a while",
 'barcelona holds huge protest in su

Tạo từ điển với mỗi chữ, từ là 1 số riêng biệt

In [ ]:
tokenizer = Tokenizer(oov_token=oov_tok, num_words=vocab_size)
tokenizer.fit_on_texts(training_sentences)

Chuyển sang dạng số

In [ ]:
training_sequences = tokenizer.texts_to_sequences(training_sentences)


In [ ]:
training_sequences

[[328, 1, 799, 3405, 2404, 47, 389, 2214, 1, 6, 2614, 8863],
 [4, 6840, 3096, 3097, 23, 2, 161, 1, 390, 2842, 6, 251, 9, 889],
 [153, 890, 2, 891, 1445, 2215, 595, 5650, 221, 133, 36, 45, 2, 8864],
 [1252, 38, 213, 382, 2, 1572, 29, 288, 23, 10, 2405, 1446, 5651, 958],
 [715, 672, 5652, 1043, 8865, 662, 553, 5, 4, 92, 1253, 90],
 [8866, 4, 366, 70],
 [4, 6841, 369, 6, 498, 3406, 1875, 1378],
 [20, 563, 36, 1091, 31, 163, 2, 103, 87, 18, 150, 6, 33, 343],
 [278, 3407, 6842, 447, 8867, 2092, 148],
 [2093, 300, 335, 370, 63, 1, 6, 4, 4264],
 [3098, 2216, 3756, 14, 37, 5653, 8868, 5, 2094, 1092],
 [309, 767, 428, 8, 1663, 1664, 9, 3099],
 [226, 477, 2843, 13, 9, 922, 239, 371, 2, 4265, 1, 6843],
 [235, 5654, 8869, 3757, 39, 240, 1, 6, 7, 174],
 [1, 1379, 800, 663, 5, 336, 3, 959],
 [524, 2094, 8870, 126, 8871, 6, 8872, 3758, 1665],
 [2095,
  1328,
  341,
  46,
  3408,
  323,
  288,
  960,
  2,
  22,
  1,
  19,
  1044,
  359,
  109,
  1447],
 [1666, 6844, 3100, 8873, 19, 5655, 1200],
 [8874

In [ ]:
training_padded = pad_sequences(training_sequences, maxlen= max_length, padding='post', truncating='post')

In [ ]:
training_padded

array([[ 328,    1,  799, ...,    0,    0,    0],
       [   4, 6840, 3096, ...,    0,    0,    0],
       [ 153,  890,    2, ...,    0,    0,    0],
       ...,
       [  79, 1729,    1, ...,    0,    0,    0],
       [  53, 5108, 4735, ...,    0,    0,    0],
       [ 312,  705,    1, ...,    0,    0,    0]], dtype=int32)

In [ ]:
training_padded.shape

(20000, 100)

Làm tương tự với bộ validation (ở đây đặt tên là test)

In [ ]:
test_sequences = tokenizer.texts_to_sequences(testing_sentences)

In [ ]:
test_sequences

[[1, 1100, 6663, 9423, 30, 1, 2439, 5, 519, 109],
 [202, 1, 8, 31, 4642, 1, 2, 854, 773],
 [18, 380, 191, 2, 915, 76, 8, 4, 3861],
 [1, 1, 299, 337, 3, 1, 1],
 [162, 4168, 2382, 6, 8296, 2627, 348, 1241],
 [44, 3450, 1906, 627],
 [404, 345, 2, 1, 12, 1693, 1, 1596, 4725, 6, 1, 5642, 2079],
 [314, 37, 1, 2, 3133, 8, 6823, 6234, 43, 730, 10, 1492, 439, 1],
 [1,
  2781,
  12,
  2790,
  136,
  45,
  2,
  2279,
  28,
  24,
  6,
  83,
  2,
  196,
  1709,
  95,
  5325,
  493,
  230],
 [26, 4, 2636, 11, 1, 1, 9426, 21, 860, 820],
 [1031, 1230, 1, 18, 136, 1503, 2, 1028],
 [262, 3377, 2, 2929, 80, 8, 5079, 1, 2079, 43, 2431],
 [20, 1, 12, 1497, 84, 7675, 4, 1, 3, 4, 2010, 440],
 [1, 1, 4, 52, 9, 3659, 21],
 [585, 2644, 1578, 2, 1, 309, 84, 29, 4450, 45, 5875, 177],
 [4, 1432, 90, 155, 450, 735, 30, 1, 4976, 1],
 [3335, 760, 277, 164, 6, 49, 54, 2, 1273, 80, 1346, 2771],
 [102, 23, 9, 544, 5, 4968, 1],
 [5077, 6821, 481, 4, 5832, 128, 75, 1192, 3813, 5, 12, 313, 1, 1, 1],
 [1, 1606, 2, 185, 946,

In [ ]:
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post', truncating='post')

In [ ]:
#maxlen dùng để giới hạn độ dài câu
#truncating: nếu câu dài hơn maxlen thì cắt đi đằng sau

In [ ]:
test_padded.shape

(6709, 100)

In [ ]:
model = Sequential()

In [ ]:
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))

In [ ]:
model.add(Flatten())

In [ ]:
# có thể thay bằng lớp GlobalAveragePooling1D

In [ ]:
model.add(Dense(10, activation='relu'))

In [ ]:
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                16010     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 176,021
Trainable params: 176,021
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(training_padded, training_labels, epochs=10, validation_data=(test_padded, testing_labels))

Epoch 1/10
625/625 [==============================] - 4s 6ms/step - loss: 0.5330 - acc: 0.7362 - val_loss: 0.4215 - val_acc: 0.8547
Epoch 2/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2466 - acc: 0.9125 - val_loss: 0.3134 - val_acc: 0.8670
Epoch 3/10
625/625 [==============================] - 2s 4ms/step - loss: 0.1155 - acc: 0.9621 - val_loss: 0.3514 - val_acc: 0.8596
Epoch 4/10
625/625 [==============================] - 3s 4ms/step - loss: 0.0537 - acc: 0.9862 - val_loss: 0.4202 - val_acc: 0.8533
Epoch 5/10
625/625 [==============================] - 2s 4ms/step - loss: 0.0238 - acc: 0.9956 - val_loss: 0.4774 - val_acc: 0.8521
Epoch 6/10
625/625 [==============================] - 2s 4ms/step - loss: 0.0119 - acc: 0.9983 - val_loss: 0.5312 - val_acc: 0.8529
Epoch 7/10
625/625 [==============================] - 2s 4ms/step - loss: 0.0072 - acc: 0.9989 - val_loss: 0.6079 - val_acc: 0.8438
Epoch 8/10
625/625 [==============================] - 3s 4ms/step - loss: 0.

In [ ]:
test_sen = ["Everyone seems normal until you get to know them."]

# FIXME: Chuyển câu thành Sequences
test_sen_sequences = tokenizer.texts_to_sequences(test_sen)
# FIXME: padding cho câu này
padded_test_sen = pad_sequences(test_sen_sequences,maxlen=max_length, padding='post', truncating='post')
# FIXME: Tiến hành dự đoán



In [ ]:
padded_test_sen

array([[ 266, 1781, 1906,  423,   18,   59,    2,  111,  219,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0]], dtype=int32)

In [ ]:
model.predict(padded_test_sen)

array([[0.8016877]], dtype=float32)

In [ ]:
# >0.5 là mỉa mai